In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score

In [2]:
with open('images_features_2d.pickle','rb') as features:

    data=pickle.load(features)

In [3]:
data['CXR1_1_IM-0001-3001.png'].shape

(1, 7, 7, 1024)

In [4]:
def fetch_tags(image_id, tags):

    new_tags=[]

    for image in image_id.to_list():

        tag=tags[image.split('_')[0]]

        if tag[0].lower()=='normal':
            new_tags.append('normal')
        else:
            new_tags.append('abnormal')

    return new_tags

path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/latest/One_Image_norm_abnorm_split/r2gen_annotations/'

def read_r2gen_annotations():
    with open(path+'annotation.json', 'rb') as f:
        full_records = json.load(f)

    splits=['train','val','test']
    train=full_records['train']
    val = full_records['val']
    test = full_records['test']
    
    
    new_records={}

    for s in splits:
        split_records = []
        records=full_records[s]
        for r in records:
            tag=tags[r['id'].split('_')[0]]
            if tag==0:
                split_records.append(r)
            else:
                continue
        new_records[s]=split_records

with open("./data/iu_xray/annotation_10.json", "w") as write_file:
    json.dump(new_records, write_file)

def load_preprocess_data():
    tags = np.load('Data_with_tags.npy', allow_pickle=True).item()
    train = pd.read_json(path+'/train/train.json')
    #print(train.shape, train.columns)
    x_train=train['image_id']

    train_tags=fetch_tags(x_train,tags)
    #print(train_tags)
    #y_train = mlb.fit_transform([train_tags])
    y_train=pd.get_dummies(train_tags)
    #print(y_train)

    #print(y_train.head())

    test = pd.read_json(path+'/test/test.json')
    x_test = test['image_id']

    test_tags=fetch_tags(x_test,tags)
    y_test = pd.get_dummies(test_tags)
    
    return x_train,y_train,x_test,y_test



In [5]:
def load_features(split):
    features = []#np.empty((0,7,7,1024))
    
    for image in split.tolist():
        #print(image)
        #features=np.hstack((features, data[image]))
        #features=np.append(features, data[image], axis=0)
        features.append(data[image])
    features=np.array(features)
    features=np.reshape(features,(-1,49,1024))
    
    return features

In [6]:
def class_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(49,1024)))
    
    model.add(layers.GlobalAveragePooling1D())
    #model.add(layers.Dense(128,activation='relu'))
    model.add(layers.LeakyReLU(alpha=0.05))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(n, activation="sigmoid"))

    model.summary()
    
    return model



In [7]:
images_train,y_train,images_test,y_test=load_preprocess_data()

In [8]:
train_features=load_features(images_train)
test_features=load_features(images_test)

In [9]:
model=class_model(2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling1d (Gl (None, 1024)              0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                65600     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 65,730
Trainable params: 65,730
Non-trainable params: 0
_________________________________________________________________


In [10]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
])

model.fit(train_features, y_train.to_numpy(), epochs=50, callbacks=[earlyStopping],validation_data=(test_features, y_test.to_numpy()), batch_size=16)

test_tags=model.predict(test_features)

Train on 5068 samples, validate on 500 samples
Epoch 1/50
5068/5068 [==============================] - 2s 324us/sample - loss: 0.6334 - precision: 0.6477 - recall: 0.6454 - val_loss: 0.5869 - val_precision: 0.6687 - val_recall: 0.6700
Epoch 2/50
5068/5068 [==============================] - 1s 174us/sample - loss: 0.5889 - precision: 0.6789 - recall: 0.6821 - val_loss: 0.5615 - val_precision: 0.6938 - val_recall: 0.7160
Epoch 3/50
5068/5068 [==============================] - 1s 177us/sample - loss: 0.5749 - precision: 0.6857 - recall: 0.6904 - val_loss: 0.5489 - val_precision: 0.7103 - val_recall: 0.7160
Epoch 4/50
5068/5068 [==============================] - 1s 173us/sample - loss: 0.5669 - precision: 0.6920 - recall: 0.6948 - val_loss: 0.5483 - val_precision: 0.7069 - val_recall: 0.7140
Epoch 5/50
5068/5068 [==============================] - 1s 173us/sample - loss: 0.5602 - precision: 0.6824 - recall: 0.6839 - val_loss: 0.5403 - val_precision: 0.7140 - val_recall: 0.7040
Epoch 6/50
50

Epoch 44/50
5068/5068 [==============================] - 1s 181us/sample - loss: 0.5066 - precision: 0.7382 - recall: 0.7382 - val_loss: 0.5301 - val_precision: 0.7220 - val_recall: 0.7220
Epoch 45/50
5068/5068 [==============================] - 1s 180us/sample - loss: 0.5082 - precision: 0.7383 - recall: 0.7382 - val_loss: 0.5243 - val_precision: 0.7220 - val_recall: 0.7220
Epoch 46/50
5068/5068 [==============================] - 1s 185us/sample - loss: 0.5082 - precision: 0.7399 - recall: 0.7397 - val_loss: 0.5196 - val_precision: 0.7280 - val_recall: 0.7280
Epoch 47/50
5068/5068 [==============================] - 1s 181us/sample - loss: 0.5103 - precision: 0.7449 - recall: 0.7451 - val_loss: 0.5311 - val_precision: 0.7140 - val_recall: 0.7140
Epoch 48/50
5068/5068 [==============================] - 1s 185us/sample - loss: 0.5039 - precision: 0.7391 - recall: 0.7390 - val_loss: 0.5477 - val_precision: 0.7160 - val_recall: 0.7160
Epoch 49/50
5068/5068 [==============================] 

In [11]:
true=np.array(y_test).argmax(axis=1)
pred=test_tags.argmax(axis=1)
# #print(sum(test_tags.argmax(axis=1)))
# y_test=y_test.idxmax(axis=1)
# print(y_test)
print(f1_score(true,pred))
print(y_test)

0.6542553191489362
     abnormal  normal
0           1       0
1           1       0
2           1       0
3           1       0
4           0       1
..        ...     ...
495         1       0
496         0       1
497         0       1
498         1       0
499         1       0

[500 rows x 2 columns]


In [12]:
m=tf.keras.metrics.Recall()
pred=test_tags.argmax(axis=1)
m.update_state(np.array(y_test),test_tags)
m.result().numpy()

0.74

In [13]:
import pickle

all_tags={}

for idx,image in enumerate(images_test.to_list()):
    all_tags[image.split('_')[0]]=pred[idx]
    
train_true=np.array(y_train).argmax(axis=1)
print(sum(pred))
for idx,image in enumerate(images_train.to_list()):
    all_tags[image.split('_')[0]]=train_true[idx]

print(all_tags['CXR63'])
with open(path+'binary_tags_chex.pkl','wb') as file:
    pickle.dump(all_tags, file, protocol=2)
    

190
0
